# Exploring Clustering Results

In [275]:

import pandas as pd
import math
import numpy as np

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import copy as cp

import seaborn as sns

import statsmodels.api as sm


from IPython.display import display

processed_path = "..\\data\\processed\\"
file_name = "p2c1_clustering_20medoids_clean.csv"

df_prd_cluster = pd.read_csv(processed_path+file_name, sep=';', encoding='utf-8').drop('Unnamed: 0',axis=1).set_index('Product Group key')

df_prd_cluster.shape


(262, 2)

## Products description

In [276]:
raw_path = "..\\data\\raw\\"
file_name = "bnd_products_desc.csv"

df_produit = pd.read_csv(raw_path+file_name, sep=';', encoding='iso8859_2')\
                    .drop(["Key","Description","Unit of Measure 2 Ratio","Unit of Measure 4 Ratio","IMBALLO","STD/PROMO","IMBALLO (Description)"],axis=1)\
                    .drop_duplicates()


#Join with clusters
product_cluster = df_produit.join(df_prd_cluster,on='Product Group key',how='inner')

product_cluster.shape

(315, 37)

In [278]:
filename = 'bnd_product_cluster_clean.csv'
product_cluster.to_csv(processed_path+filename,sep=';',encoding='iso8859_2')